# generating normal log files for checking error

In [1]:
from pyspark import SparkContext, SparkConf #spark

In [2]:
import pydoop.hdfs as hdfs #for saving local file as hdfs 

In [3]:
from pyspark.sql import SQLContext # for data frame

In [4]:
import urllib #for imitating hdfs auto escape string

In [5]:
import re #for escape string 

In [26]:
 from operator import methodcaller

In [6]:
conf = SparkConf().setAppName("error_machine")

In [7]:
sc = SparkContext(conf=conf)

# upload file to hdfs

##### check result:
* [hdfs@ambari kristen]$ hdfs dfs -ls /user/spark/input
*  Found 2 items
*  -rw-r--r--   3 root root     213152 2017-03-08 05:06 /user/spark/input/log_error
*  -rw-r--r--   3 root root    1363076 2017-03-08 05:04 /user/spark/input/log_normal


In [5]:
from_path = './log_normal'

In [6]:
to_path ='hdfs://ambari:8020/user/spark/input'

In [7]:
hdfs.put(from_path, to_path)

IOError: '/user/spark/input/log_normal' already exists

In [ ]:
from_path = './log_error'

In [ ]:
to_path ='hdfs://ambari:8020/user/spark/input'

In [ ]:
hdfs.put(from_path, to_path)

# wordcount by each text but special characters

In [113]:
inputPath = '/user/spark/input/log_normal'

In [114]:
in_hdfs = sc.textFile(inputPath)

In [115]:
count_lines = in_hdfs.count()
print count_lines

11364


In [181]:
# spliting by special characters and number
rdd_split_map = in_hdfs.flatMap(lambda x: re.split(r'[`\- =~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?0-9]', x),1)

In [182]:
rdd_split_map.take(100)

[u'Jan',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'jerry',
 u'virtual',
 u'machine',
 u'anacron',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'Job',
 u'',
 u'cron',
 u'daily',
 u'',
 u'terminated',
 u'Jan',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'jerry',
 u'virtual',
 u'machine',
 u'anacron',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'Normal',
 u'exit',
 u'',
 u'',
 u'',
 u'job',
 u'run',
 u'',
 u'Jan',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'jerry',
 u'virtual',
 u'machine',
 u'dhclient',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'DHCPREQUEST',
 u'of',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'']

In [183]:
#ready to count
rdd_count_map = rdd_split_map.map(lambda x: (x, 1))

In [184]:
rdd_count_map.take(10)

[(u'Jan', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1)]

In [185]:
#mapping each key and counting it
res_reduce = rdd_count_map.reduceByKey(lambda a, b: a+b )

In [187]:
# desc sorting by counting number
res_reduce = res_reduce.sortBy(lambda c: c[1], False)
res_reduce.take(10)

[(u'', 483648),
 (u'virtual', 11546),
 (u'jerry', 11365),
 (u'Jan', 11364),
 (u'machine', 11364),
 (u'NetworkManager', 4390),
 (u'info', 4327),
 (u'dhclient', 2483),
 (u'dispatcher', 2137),
 (u'nm', 2137)]

# Picking counting >= lines -2
* Q -2? A: some case ----- or ... show in the last lines

In [188]:
#picking string which always show and counting bigger than file lines  (deviation value 2)
pick_top_string_map = res_reduce.filter(lambda x: x[1] >= count_lines-2)

In [189]:
pick_top_string_map.collect()

[(u'', 483648),
 (u'virtual', 11546),
 (u'jerry', 11365),
 (u'Jan', 11364),
 (u'machine', 11364)]

# Picking counting < lines -2

In [190]:
pick_last_string_map = res_reduce.filter(lambda x: x[1] < count_lines-2)

In [191]:
pick_last_string_map.sortBy(lambda c: c[1], False).take(3)

[(u'NetworkManager', 4390), (u'info', 4327), (u'dhclient', 2483)]

# Deleting those top  texts in each line

In [359]:
list_tops = pick_top_string_map.collect()

In [360]:
for top in list_tops:
    print top[0]


virtual
jerry
Jan
machine


In [560]:
# spliting by special characters
rdd_sp_map = in_hdfs.map(lambda x: re.split(r'[`\- =~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?0-9]', x),1)

In [561]:
rdd_sp_map.take(1)


[[u'Jan',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'jerry',
  u'virtual',
  u'machine',
  u'anacron',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'Job',
  u'',
  u'cron',
  u'daily',
  u'',
  u'terminated']]

In [562]:
def Filter_Top_Texts(line, list_tops):
    res_str = ""
    #filter all the top text 
    for text in line:
        flag = 0
        if str(text) == "":
            continue;
        for top in list_tops:
            #res_str +='----' +top[0]
            if top[0] == str(text):

                flag = 1
                break;
        if(flag == 1):
            continue;
        else:
            res_str +="," + str(text)
    return res_str[1:]

In [563]:
#filter all the top text 
rdd_fil_map = rdd_sp_map.map(lambda x:Filter_Top_Texts(x, list_tops) )

In [564]:
rdd_fil_map.take(2)

['anacron,Job,cron,daily,terminated', 'anacron,Normal,exit,job,run']

# map reduce each line by top key (counting < lines -2)

In [565]:
last_tops = pick_last_string_map.collect()

In [566]:
for top in last_tops:
    print top

(u'NetworkManager', 4390)
(u'info', 4327)
(u'dhclient', 2483)
(u'dispatcher', 2137)
(u'nm', 2137)
(u'ens', 2044)
(u'systemd', 1992)
(u'to', 1579)
(u'bound', 1574)
(u'dhcp', 1573)
(u'of', 1480)
(u'org', 1359)
(u'freedesktop', 1341)
(u'service', 1290)
(u'dbus', 1281)
(u'x', 952)
(u'on', 945)
(u'Started', 933)
(u'd', 886)
(u'in', 868)
(u'change', 866)
(u'Service', 858)
(u'port', 857)
(u'req', 856)
(u'scripts', 856)
(u'Manager', 855)
(u'name', 855)
(u'veth', 855)
(u'system', 855)
(u'Network', 854)
(u'Dispatcher', 854)
(u'Script', 854)
(u'from', 771)
(u'seconds', 746)
(u'xid', 726)
(u'renewal', 724)
(u'DHCPREQUEST', 724)
(u'DHCPACK', 724)
(u'time', 691)
(u'kernel', 641)
(u'state', 583)
(u'b', 581)
(u'f', 563)
(u'address', 555)
(u'e', 543)
(u'new', 511)
(u'nameserver', 508)
(u'device', 489)
(u'c', 481)
(u'CUPS', 469)
(u'Starting', 459)
(u'Scheduler', 457)
(u'Successfully', 448)
(u'start', 435)
(u'activated', 433)
(u'running', 433)
(u'server', 433)
(u'request', 432)
(u'ordered', 428)
(u'chang

In [567]:
pick_last_string_map.take(2)

[(u'NetworkManager', 4390), (u'info', 4327)]

In [568]:
def Group_Map_by_top_key(line):
    res_str = ""
    find_key = ""
    flag = 0
    #line = line.split(',')
    #for text in line:
        #res_str += "%%%%%%% "+text +"%%%%"
    #filter all the top text 
    for top in last_tops:
        flag = 0
        for text in line:
            #res_str +='----' +top[0]
            if str(top[0]) == str(text):
                #res_str +="*** "+ str(top[0])+"*** "+ str(text)
                find_key= str(text)
                flag = 1
                break;
        if(flag == 1):
            break;
    if(flag == 1):
        for text in line:
            if(find_key != text):
                res_str +="," + str(text)
    return (find_key,res_str[1:])

In [569]:
#map by key
rdd_key_map = rdd_fil_map.map(lambda x:Group_Map_by_top_key(x.split(',')) )

In [570]:
rdd_key_map.take(2)

[('cron', 'anacron,Job,daily,terminated'), ('exit', 'anacron,Normal,job,run')]

In [571]:
rdd_key_red = rdd_key_map.reduceByKey(lambda a,b:a+b)

In [572]:
rdd_key_red.take(1)

[('info',
  'dockerd,time,T,level,msg,libcontainerd,new,containerd,process,piddockerd,time,T,level,msg,Graph,migration,to,content,addressability,took,secondsdockerd,time,T,level,msg,Loading,containers,startdockerd,time,T,level,msg,Firewalld,running,falsedockerd,time,T,level,msg,Default,bridge,docker,is,assigned,with,an,IP,address,Daemon,option,bip,can,be,used,to,set,a,preferred,IP,addressdockerd,time,T,level,msg,Loading,containers,donedockerd,time,T,level,msg,Daemon,has,completed,initializationdockerd,time,T,level,msg,Docker,daemon,commit,cf,graphdriver,aufs,versiondockerd,time,T,level,msg,API,listen,on,var,run,docker,sockdockerd,time,T,level,msg,Layer,sha,b,bdf,f,e,f,c,b,e,c,b,ef,ba,a,a,ba,cleaned,updockerd,time,T,level,msg,No,non,localhost,DNS,nameservers,are,left,in,resolv,conf,Using,default,external,servers,nameserver,nameserverdockerd,time,T,level,msg,IPv,enabled,Adding,default,IPv,external,servers,nameserver,nameserverdockerd,time,T,level,msg,No,non,localhost,DNS,nameservers,are,

In [573]:
#distinct value
list_temp = []
rdd_collect = rdd_key_red.collect()
for v in rdd_collect:
    rdd_temp = sc.parallelize(v[1:][0].split(','))
    rdd_dis_val_map = rdd_temp.map(lambda x: (x,1))
    rdd_dis_val_red = rdd_dis_val_map.reduceByKey(lambda a,b: a+b)
    rdd_dis_fn = rdd_dis_val_red.map(lambda x: x[0])
    #print rdd_dis_fn.collect()
    list_temp.append((v[0],rdd_dis_fn.collect()))
print list_temp[:1]


[('info', ['ac', 'ae', 'ff', 'process', 'enabled', 'is', 'ea', 'Daemon', 'ec', 'assigned', 'conf', 'updockerd', 'aeff', 'terminateddockerd', 'bfe', 'nameservers', 'addressdockerd', 'content', 'cc', 'fada', 'msg', 'containers', 'dde', 'non', 'run', 'Default', 'completed', 'db', 'daemon', 'preferred', 'var', 'running', 'IPv', 'Layer', 'DNS', 'Docker', 'with', 'docker', 'bip', 'a', 'c', 'e', 'default', 'signal', 'bef', 'fb', 'startdockerd', 'versiondockerd', 'localhost', 'stopping', 'fd', 'nameserver', 'addressability', 'Adding', 'Loading', 'receiving', 'ad', 'ead', 'No', 'Graph', 'fe', 'ef', 'cf', 'sockdockerd', 'servers', 'set', 'migration', 'dockerd', 'initializationdockerd', 'aufs', 'after', 'option', 'are', 'terminated', 'bdf', 'listen', 'bfd', 'containerd', 'left', 'Processing', 'to', 'libcontainerd', 'eaeba', 'Using', 'piddockerd', 'ddd', 'bridge', 'be', 'used', 'an', 'ba', 'dcfddf', 'IP', 'de', 'Firewalld', 'fc', 'took', 'fa', 'b', 'cleaned', 'T', 'new', 'falsedockerd', 'ab', 'nam

# compare error log

In [574]:
inputPath_error = '/user/spark/input/log_error'

In [575]:
df_error = sc.textFile(inputPath_error)

In [576]:
# spliting by special characters and number
rdd_split_map_error = df_error.flatMap(lambda x: re.split(r'[`\- =~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?0-9]', x),1)

In [577]:
rdd_split_map_error.take(50)

[u'Jan',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'jerry',
 u'virtual',
 u'machine',
 u'anacron',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'Job',
 u'',
 u'cron',
 u'daily',
 u'',
 u'terminated',
 u'Jan',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'',
 u'jerry',
 u'virtual',
 u'machine',
 u'anacron',
 u'',
 u'']

In [578]:
#ready to count
rdd_count_map_error = rdd_split_map.map(lambda x: (x, 1))

In [579]:
rdd_count_map_error.take(10)

[(u'Jan', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1),
 (u'', 1)]

In [580]:
#mapping each key and counting it
res_reduce_error = rdd_count_map_error.reduceByKey(lambda a, b: a+b )

In [581]:
# desc sorting by counting number
res_reduce = res_reduce_error.sortBy(lambda c: c[1], False)
res_reduce.take(10)

[(u'', 483648),
 (u'virtual', 11546),
 (u'jerry', 11365),
 (u'Jan', 11364),
 (u'machine', 11364),
 (u'NetworkManager', 4390),
 (u'info', 4327),
 (u'dhclient', 2483),
 (u'dispatcher', 2137),
 (u'nm', 2137)]

In [582]:
#picking string which always show and counting bigger than file lines  (deviation value 2)
pick_top_string_map = res_reduce.filter(lambda x: x[1] >= count_lines-2)

In [583]:
pick_top_string_map.take(5)

[(u'', 483648),
 (u'virtual', 11546),
 (u'jerry', 11365),
 (u'Jan', 11364),
 (u'machine', 11364)]

In [584]:
pick_last_string_map = res_reduce.filter(lambda x: x[1] < count_lines-2)

In [585]:
pick_last_string_map.sortBy(lambda c: c[1], False).take(3)

[(u'NetworkManager', 4390), (u'info', 4327), (u'dhclient', 2483)]

In [586]:
list_tops = pick_top_string_map.collect()

In [587]:
for top in list_tops:
    print top[0]


virtual
jerry
Jan
machine


In [588]:
# spliting by special characters
rdd_map_error = df_error.map(lambda x: re.split(r'[`\- =~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?0-9]', x),1)

In [589]:
rdd_map_error.take(1)

[[u'Jan',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'jerry',
  u'virtual',
  u'machine',
  u'anacron',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'',
  u'Job',
  u'',
  u'cron',
  u'daily',
  u'',
  u'terminated']]

In [590]:
#filter all the top text 
rdd_fil_map = rdd_map_error.map(lambda x:Filter_Top_Texts(x, list_tops) )

In [591]:
rdd_fil_map.take(4)

['anacron,Job,cron,daily,terminated',
 'anacron,Normal,exit,job,run',
 'dhclient,DHCPREQUEST,of,on,ens,to,port,xid,x,ce,aa',
 'dhclient,DHCPACK,of,from']

In [592]:
last_tops = pick_last_string_map.collect()

In [593]:
for top in last_tops:
    print top

(u'NetworkManager', 4390)
(u'info', 4327)
(u'dhclient', 2483)
(u'dispatcher', 2137)
(u'nm', 2137)
(u'ens', 2044)
(u'systemd', 1992)
(u'to', 1579)
(u'bound', 1574)
(u'dhcp', 1573)
(u'of', 1480)
(u'org', 1359)
(u'freedesktop', 1341)
(u'service', 1290)
(u'dbus', 1281)
(u'x', 952)
(u'on', 945)
(u'Started', 933)
(u'd', 886)
(u'in', 868)
(u'change', 866)
(u'Service', 858)
(u'port', 857)
(u'req', 856)
(u'scripts', 856)
(u'Manager', 855)
(u'name', 855)
(u'veth', 855)
(u'system', 855)
(u'Network', 854)
(u'Dispatcher', 854)
(u'Script', 854)
(u'from', 771)
(u'seconds', 746)
(u'xid', 726)
(u'renewal', 724)
(u'DHCPREQUEST', 724)
(u'DHCPACK', 724)
(u'time', 691)
(u'kernel', 641)
(u'state', 583)
(u'b', 581)
(u'f', 563)
(u'address', 555)
(u'e', 543)
(u'new', 511)
(u'nameserver', 508)
(u'device', 489)
(u'c', 481)
(u'CUPS', 469)
(u'Starting', 459)
(u'Scheduler', 457)
(u'Successfully', 448)
(u'start', 435)
(u'activated', 433)
(u'running', 433)
(u'server', 433)
(u'request', 432)
(u'ordered', 428)
(u'chang

In [594]:
pick_last_string_map.take(2)

[(u'NetworkManager', 4390), (u'info', 4327)]

In [595]:
#map by key
rdd_key_map = rdd_fil_map.map(lambda x:Group_Map_by_top_key(x.split(',')) )

In [596]:
rdd_key_map.take(3)

[('cron', 'anacron,Job,daily,terminated'),
 ('exit', 'anacron,Normal,job,run'),
 ('dhclient', 'DHCPREQUEST,of,on,ens,to,port,xid,x,ce,aa')]

In [597]:
rdd_key_red = rdd_key_map.reduceByKey(lambda a,b:a+b)

In [598]:
rdd_key_red.take(1)

[('systemd',
  'Starting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Network,Manager,Script,Dispatcher,ServiceStarted,Network,Manager,Script,Dispatcher,ServiceStarted,CUPS,SchedulerStarting,Automatically,refresh,installed,snapsStarted,Automatically,refresh,installed,snapssnapd,refresh,timer,Adding,h,min,s,random,time

In [619]:
outputFolder = "/user/spark/output"

In [620]:
#distinct value
list_temp_error = {}
rdd_collect = rdd_key_red.collect()
for v in rdd_collect:
    rdd_temp = sc.parallelize(v[1:][0].split(','))
    rdd_dis_val_map = rdd_temp.map(lambda x: (x,1))
    rdd_dis_val_red = rdd_dis_val_map.reduceByKey(lambda a,b: a+b)
    rdd_dis_fn = rdd_dis_val_red.map(lambda x: x[0])
    #print rdd_dis_fn.collect()
    list_temp_error[v[0]] = rdd_dis_fn.collect()
    rdd_dis_fn.saveAsTextFile(outputFolder +"/"+" error_case"+"/"+ v[0])
print list_temp_error.items()[0]

('kernel', ['rawserio', 'sysfillrectsysfillrect', 'ffff', 'userxfrm', 'stpstp', 'pvscsi', 'llc', 'intelkvm', 'pcnetpcnet', 'nfitnfit', 'PID', 'fb', 'Read', 'llcllc', 'lp', 'kms', 'CRCR', 'GS', 'cffff', 'masquerade', 'MASQUERADEipt', 'vhostmacvtap', 'hid', 'EFLAGSRIP', 'desd', 'vmxnetvmxnetvmxnethid', 'genericusbhidusbhidusbhidhidhidhidvmwgfxvmwgfxvmwgfx', 'leds', 'RDI', 'tablesip', 'ffffea', 'Traceffff', 'soft', 'netfilter', 'CRR', 'seq', 'ee', 'cbWorkqueue', 'fFS', 'snd', 'ec', 'ffffRSP', 'transport', 'kthread', 'rv', 'xor', 'power', 'for', 'eStackStackffffffffc', 'iptable', 'exit', 'coretempcoretemp', 'net', 'parportparport', 'bd', 'hfsplus', 'timersnd', 'autofs', 'G', 'irqbypassirqbypass', 'pata', 'MASQUERADEnf', 'SUCCESS', 'br', 'sndsnd', 'ffffffffc', 'ffcc', 'ES', 'sys', 'c', 'gameportgameport', 'seqsnd', 'tablesvmw', 'timer', 'filter', 's', 'drm', 'jfs', 'UbuntuCPU', 'mptspimptspi', 'mptscsihmptscsih', 'tcpudp', 'ppdevppdevppdev', 'spimptbase', 'xd', 'fbc', 'device', 'devicesnd',

In [ ]:
list_temp_error

In [608]:
a= {}
a["ww"] = 'sd'
a['ee'] = 'aa'
print a

{'ww': 'sd', 'ee': 'aa'}


In [614]:
print a.items()[0]

('ww', 'sd')
